# Scratch

---

### Imports & Setup

Installs

In [1]:
# Run this cell to install required packages in your Jupyter notebook environment
!pip install openai sqlalchemy sqlmodel psycopg2-binary pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 60.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


Connect to DB

In [2]:
from sqlalchemy import create_engine
from sqlmodel import Session, SQLModel, select

# PostgreSQL connection URL
DATABASE_HOST = "db"  # Service name in docker-compose.yml
DATABASE_PORT = "5432"
DATABASE_NAME = "updatesdb"
DATABASE_USER = "postgres"
DATABASE_PASSWORD = "your_password"

ENGINE_URL = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"

engine = create_engine(ENGINE_URL, echo=True, pool_pre_ping=True)

# Optional: If you need to create the tables in your database (uncomment if necessary)
# SQLModel.metadata.create_all(engine)

Models

In [3]:
from typing import Optional
from sqlmodel import SQLModel, Field, Relationship
from sqlalchemy import Column, BigInteger

class ChatType(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    type_name: str = Field(sa_column_kwargs={"unique": False})

class Chat(SQLModel, table=True):
    id: int = Field(primary_key=True)
    chat_id: Optional[int] = Field(default=None, sa_column=Column(BigInteger()))
    all_members_are_administrators: bool = Field()
    title: str = Field()
    type_id: int = Field(foreign_key="chattype.id")
    type: ChatType = Relationship()

class User(SQLModel, table=True):
    id: int = Field(primary_key=True, index=True)
    user_id: Optional[int] = Field(default=None)  # Telegram user ID
    first_name: str = Field()
    last_name: Optional[str] = Field(default=None)
    is_bot: bool = Field()
    language_code: str = Field()
    username: Optional[str] = Field()

class Message(SQLModel, table=True):
    id: int = Field(primary_key=True)
    message_id: int = Field(default=None)
    channel_chat_created: bool = Field()
    chat_id: int = Field(foreign_key="chat.id")
    chat: Chat = Relationship()
    date: int = Field()
    delete_chat_photo: bool = Field()
    from_user_id: int = Field(foreign_key="user.id")
    from_user: User = Relationship()
    group_chat_created: bool = Field()
    reply_to_message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    reply_to_message: 'Message' = Relationship(sa_relationship_kwargs={"remote_side": "Message.id"})
    supergroup_chat_created: bool = Field()
    text: Optional[str] = Field()

class Update(SQLModel, table=True):
    update_id: int = Field(primary_key=True)
    message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    message: Optional[Message] = Relationship()

---

### Display Tables (in pandas)

In [4]:
import pandas as pd

def display_table_data_pandas(model_class):
    with Session(engine) as session:
        statement = select(model_class)
        results = session.exec(statement).all()
        return pd.DataFrame([result.__dict__ for result in results])

In [5]:
# Display data using pandas for better formatting
data = display_table_data_pandas(Message)
data.head()

2024-05-10 12:53:46,018 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-10 12:53:46,018 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-10 12:53:46,019 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-10 12:53:46,019 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-10 12:53:46,020 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-10 12:53:46,020 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-10 12:53:46,021 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-10 12:53:46,027 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message
2024-05-10 12:53:46,027 INFO sqlalchemy.engine.Engine [generated in 0.00032s] {}
2024-05-10 12:53:46,028 INFO sqlalchemy.engine.Engine ROLLBACK


,_sa_instance_state,message_id,chat_id,date,from_user_id,group_chat_created,id,channel_chat_created,delete_chat_photo,reply_to_message_id,supergroup_chat_created,text
0,<sqlalchemy.orm.state.InstanceState object at ...,22497,2,1715268507,2,False,1,False,False,NaN,False,are you saying this due to the whole wallet wh...
1,<sqlalchemy.orm.state.InstanceState object at ...,22501,1,1715270079,1,False,2,False,False,1.0,False,No. the same whitelisting issues exists with 7...
2,<sqlalchemy.orm.state.InstanceState object at ...,248,3,1715270331,3,False,3,False,False,NaN,False,"Ok Archiving is working fine for now, will sta..."
3,<sqlalchemy.orm.state.InstanceState object at ...,22502,4,1715270488,4,False,4,False,False,NaN,False,"From the POV of wallets, I think the whitelist..."
4,<sqlalchemy.orm.state.InstanceState object at ...,22502,6,1715270488,6,False,5,False,False,NaN,False,"From the POV of wallets, I think the whitelist..."


---

### Reconstruct Chat History

In [6]:
from datetime import datetime, timedelta

def reconstruct_chat_as_text(engine, start_time: datetime, end_time: datetime):
    chat_history = []
    with Session(engine) as session:
        # Define the query with a time filter and order by date
        statement = select(Message, User).join(User).where(
            (Message.date >= start_time.timestamp()) & 
            (Message.date <= end_time.timestamp())
        ).order_by(Message.date)

        results = session.exec(statement).all()

        for message, user in results:
            if message.reply_to_message_id:
                # Fetch the original message and user if the current message is a reply
                original_message = session.get(Message, message.reply_to_message_id)
                original_user = session.get(User, original_message.from_user_id)
                chat_line = f"{user.username}: <replying to: {original_user.username}: {original_message.text}> {message.text}"
            else:
                chat_line = f"{user.username}: {message.text}"
            
            chat_history.append(chat_line)

    return "\n".join(chat_history)

In [11]:
# Get dates
today = datetime.now()
previous_day = today - timedelta(days=1)
# Set start_time to the beginning of the previous day (00:00:00)
start_time = datetime(previous_day.year, previous_day.month, previous_day.day, 0, 0, 0)
# Set end_time to the end of the previous day (23:59:59)
end_time = datetime(previous_day.year, previous_day.month, previous_day.day, 23, 59, 59)

# Get chat history text
chat_text = reconstruct_chat_as_text(engine, start_time, end_time)

2024-05-10 12:56:10,410 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-10 12:56:10,411 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text, "user".id AS id_1, "user".user_id, "user".first_name, "user".last_name, "user".is_bot, "user".language_code, "user".username 
FROM message JOIN "user" ON "user".id = message.from_user_id 
WHERE message.date >= %(date_1)s AND message.date <= %(date_2)s ORDER BY message.date
2024-05-10 12:56:10,411 INFO sqlalchemy.engine.Engine [cached since 51.88s ago] {'date_1': 1715212800.0, 'date_2': 1715299199.0}
2024-05-10 12:56:10,413 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
print(chat_text)

julrach: are you saying this due to the whole wallet whitelisting debate?
drortirosh: <replying to: julrach: are you saying this due to the whole wallet whitelisting debate?> No. the same whitelisting issues exists with 7702 and 3074: You still fully trust your wallet to decide what code is safe to sign and what not.

The fact a single transaction signed by account "X" can use and modify many other accounts which may (or may not) attempt to submit transactions of their own - this causes a problem for block-builder to determine the validity of those transaction. They have to simulate them completely, before deciding some should be dropped. Carefully crafted transactions can cause a block builder to spend a lot of time handling and dropping transactions, to a point it can't create an optimal block.
These problems are mitigated by 7702, since you can tell those side-effects by examining the TX themselves, without executing them.
Ramshreyas: Ok Archiving is working fine for now, will start

---

### Extract topics

In [42]:
from openai import OpenAI

client = OpenAI(api_key="your_api_key_here")

def extract_topics_from_chat(chat_text):
    # Using the chat-specific completion API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Use an appropriate model, like 'gpt-3.5-turbo'
        messages=[{"role": "system", "content": "Extract the main topics from the following group chat:"},
                  {"role": "user", "content": chat_text}],
        max_tokens=300,
        temperature=0.5,
        stop=["\n\n"]
    )
    
    return response.choices[0].message.content

In [43]:
# Assume the OpenAI client is initialized and `extract_topics_from_chat` is defined
dummy_chat_text = """
Alice: I just got a new kitten yesterday!
Bob: Oh, that's awesome! What breed?
Alice: She's a Maine Coon, absolutely adorable.
Charlie: Speaking of pets, I've been thinking about getting a fish tank.
Alice: Fish are cool, but they seem like a lot of work.
Bob: <replying to: Charlie: Speaking of pets, I've been thinking about getting a fish tank.> It's not too bad, you should definitely go for it. I had one as a kid.
Charlie: Any recommendations for beginner fish?
Bob: Goldfish are pretty straightforward, but bettas have more personality.
Alice: <replying to: Bob: Oh, that's awesome! What breed?> Cats are easier to handle though, don't you think?
Bob: <replying to: Alice: Cats are easier to handle though, don't you think?> Definitely, cats are more independent.
Charlie: I like dogs more though. They're more interactive.
Alice: I'm allergic to dogs unfortunately.
Bob: <replying to: Charlie: I like dogs more though. They're more interactive.> Dogs are great for active people. Always ready to go out.
Charlie: <replying to: Bob: Goldfish are pretty straightforward, but bettas have more personality.> I might consider a betta then.
Alice: Just make sure the tank is big enough, they need space to thrive.
Bob: <replying to: Alice: Just make sure the tank is big enough, they need space to thrive.> Yeah, and keep the water clean!
"""

topics = extract_topics_from_chat(dummy_chat_text)

In [44]:
print(topics)

1. Pets (kitten, fish, cats, dogs)
2. Fish tanks and fish care (beginner fish, tank size, water cleanliness)
3. Different pet preferences (cats, dogs, fish)
4. Allergies to pets
5. Pet ownership responsibilities and considerations
